In [1]:
# %pip install nbformat
# %pip install ipywidgets
# %pip install yfinance
# %pip install seaborn
# %pip install scipy
# %pip install matplotlib
# %pip install scikit-learn
# %pip install scikit-optimize
# %pip install joblib
# %pip install plotly
# %pip install jupyter ipython django-extensions
# %pip install pandas-profiling
# %pip install pandas
# %pip install pyspark
# %pip install numpy
# %pip install pandas-datareader
# %pip install tabulate
# %pip install ta
# %pip install lime
# %pip install shap
# %pip install keras
# %pip install tensorflow

In [11]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import yfinance as yf
 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV

# sklearn Regressor Models
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

# Data Preparation
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures

# import Visualisation library
from tabulate import tabulate

import ta
from sklearn.decomposition import PCA

# import shap
# import lime.lime_tabular

import warnings
warnings.filterwarnings(
    'ignore', 'invalid value encountered in double_scalars')


# Discussion 1: Determine the best period (time) for model training.

In [12]:
# Define the stock symbol
stock_symbol = 'GOOGL'

# Set different start dates
start_dates = ['2012-01-01', '2013-01-01', '2014-01-01',
               '2015-01-01', '2016-01-01', '2017-01-01',
               '2018-01-01', '2019-01-01', '2020-01-01', '2021-01-01']

# Set end date for historical price data
end_date = '2023-05-31'

# Set empty lists to store evaluation metrics
r2_svr_train_list = []
r2_svr_test_list = []
mse_svr_train_list = []
mse_svr_test_list = []
mae_svr_train_list = []
mae_svr_test_list = []

r2_rfr_train_list = []
r2_rfr_test_list = []
mse_rfr_train_list = []
mse_rfr_test_list = []
mae_rfr_train_list = []
mae_rfr_test_list = []

for i, start_date in enumerate(start_dates):
    # Download historical price data from Yahoo Finance and store in a pandas DataFrame for each start date
    # [Close, High, Low, Open, Adj Close]
    df = yf.download(stock_symbol, start=start_date,
                     end=end_date, progress=False)
    
    # Add technical indicators
    # Simple Moving Average (SMA)
    df['SMA_10'] = ta.trend.SMAIndicator(df['Close'], window=10).sma_indicator()

    # Exponential Moving Average (EMA)
    df['EMA_10'] = ta.trend.EMAIndicator(df['Close'], window=10).ema_indicator()

    # Relative Strength Index (RSI)
    df['RSI'] = ta.momentum.RSIIndicator(df['Close'], window=10).rsi()

    # Average True Range (ATR)
    df['ATR'] = ta.volatility.AverageTrueRange(
        df['High'], df['Low'], df['Close'], window=14)

    # Moving Average Convergence Divergence (MACD)
    macd = ta.trend.MACD(df['Close'], window_slow=26,
                        window_fast=12, window_sign=9)
    df['MACD'] = macd.macd() 
    df['MACD_Signal'] = macd.macd_signal()

    # Add lag features, remove missing values, and scale the features
    lag_periods = 30  # Number of lag periods for the lag features

    # Add lag features
    for i in range(1, lag_periods + 1):
        df.loc[:, f'lag_{i}'] = df['Close'].shift(i).values.copy()

    # Remove rows with missing values
    df.dropna(inplace=True)

    # Define your independent variables (features)
    X = df.drop(['Close','Adj Close'], axis=1)
    y = df['Close']

    # Perform feature selection by selecting features with a correlation coefficient of at least 0.5 with the target variable
    corr = X.corrwith(y)
    corr_threshold = 0.5
    selected_features = corr[abs(corr) > corr_threshold].index.tolist()
    X = X[selected_features]

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=10, shuffle=False)

    # Preprocess the data by scaling it
    scaler = MinMaxScaler(feature_range=(0, 1))
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Build SVR model
    svr_model = SVR()
    svr_model.fit(X_train_scaled, y_train)

    # # Build RFR model
    rfr_model = RandomForestRegressor()
    rfr_model.fit(X_train_scaled, y_train)

    # Test the models and evaluate the performance metrics
    # Calculate evaluation metrics for SVR model
    y_svr_pred_train = svr_model.predict(X_train_scaled)
    r2_svr_train = r2_score(y_train, y_svr_pred_train)
    mse_svr_train = mean_squared_error(y_train, y_svr_pred_train)
    mae_svr_train = mean_absolute_error(y_train, y_svr_pred_train)

    y_svr_pred_test = svr_model.predict(X_test_scaled)
    r2_svr_test = r2_score(y_test, y_svr_pred_test)
    mse_svr_test = mean_squared_error(y_test, y_svr_pred_test)
    mae_svr_test = mean_absolute_error(y_test, y_svr_pred_test)
    
    r2_svr_train_list.append(r2_svr_train)
    mse_svr_train_list.append(mse_svr_train)
    mae_svr_train_list.append(mae_svr_train)

    r2_svr_test_list.append(r2_svr_test)
    mse_svr_test_list.append(mse_svr_test)
    mae_svr_test_list.append(mae_svr_test)

    # Calculate evaluation metrics for RFR model
    y_rfr_pred_train = rfr_model.predict(X_train_scaled)
    r2_rfr_train = r2_score(y_train, y_rfr_pred_train)
    mse_rfr_train = mean_squared_error(y_train, y_rfr_pred_train)
    mae_rfr_train = mean_absolute_error(y_train, y_rfr_pred_train)

    y_rfr_pred_test = rfr_model.predict(X_test_scaled)
    r2_rfr_test = r2_score(y_test, y_rfr_pred_test)
    mse_rfr_test = mean_squared_error(y_test, y_rfr_pred_test)
    mae_rfr_test = mean_absolute_error(y_test, y_rfr_pred_test)
    
    r2_rfr_train_list.append(r2_rfr_train)
    mse_rfr_train_list.append(mse_rfr_train)
    mae_rfr_train_list.append(mae_rfr_train)
    r2_rfr_test_list.append(r2_rfr_test)
    mse_rfr_test_list.append(mse_rfr_test)
    mae_rfr_test_list.append(mae_rfr_test)

svr_table = []
for i in range(len(start_dates)):
    year = start_dates[i][:4]
    svr_row = [year, r2_svr_train_list[i], r2_svr_test_list[i], mse_svr_train_list[i],
            mse_svr_test_list[i], mae_svr_train_list[i], mae_svr_test_list[i]]
    svr_table.append(svr_row)

print("SVR Model")
print(tabulate(svr_table, headers=["Year", "R² (Train)", "R² (Test)",
      "MSE (Train)", "MSE (Test)", "MAE (Train)", "MAE (Test)"], tablefmt="fancy_grid"))

rfr_table = []
for i in range(len(start_dates)):
    year = start_dates[i][:4]
    rfr_row = [year, r2_rfr_train_list[i], r2_rfr_test_list[i], mse_rfr_train_list[i],
            mse_rfr_test_list[i], mae_rfr_train_list[i], mae_rfr_test_list[i]]
    rfr_table.append(rfr_row)

print("RFR Model")
print(tabulate(rfr_table, headers=["Year", "R² (Train)", "R² (Test)",
      "MSE (Train)", "MSE (Test)", "MAE (Train)", "MAE (Test)"], tablefmt="fancy_grid"))


SVR Model
╒════════╤══════════════╤═════════════╤═══════════════╤══════════════╤═══════════════╤══════════════╕
│   Year │   R² (Train) │   R² (Test) │   MSE (Train) │   MSE (Test) │   MAE (Train) │   MAE (Test) │
╞════════╪══════════════╪═════════════╪═══════════════╪══════════════╪═══════════════╪══════════════╡
│   2012 │     0.99289  │  -10.8049   │       2.70958 │    3711.24   │      0.667792 │     53.6302  │
├────────┼──────────────┼─────────────┼───────────────┼──────────────┼───────────────┼──────────────┤
│   2013 │     0.992511 │   -5.81265  │       3.13274 │    2259.44   │      0.777998 │     36.4799  │
├────────┼──────────────┼─────────────┼───────────────┼──────────────┼───────────────┼──────────────┤
│   2014 │     0.994821 │   -1.90803  │       2.6109  │    1061.82   │      0.864832 │     22.4195  │
├────────┼──────────────┼─────────────┼───────────────┼──────────────┼───────────────┼──────────────┤
│   2015 │     0.991784 │    0.692576 │       5.39912 │     108.066  │  

Ans: SVR model is good with data starting from 2012, while RFR model is good with data starting from 2015. Overall, I will continue with Year 2015, since both model can perform prediction.

# Interepret Library

## SHAP

In [ ]:
# feature_names = poly.get_feature_names_out(X.columns)

# def ensemble_predict(X):
#     preds = ensemble_model.predict(X)
#     return preds

# explainer = shap.Explainer(ensemble_predict, X_train_poly)
# shap_values = explainer(X_test_poly)

# # Plot SHAP summary plot
# shap.summary_plot(shap_values, X_test_poly, feature_names=feature_names)

## Lime

In [ ]:

# # assuming 'ensemble_model' is your trained regression ensemble model
# def predict(X):
#     preds = ensemble_model.predict(X)
#     return preds


# explainer = lime.lime_tabular.LimeTabularExplainer(training_data=X_train_poly,
#                                                    mode='regression',
#                                                    feature_names=feature_names,
#                                                    class_names=None,
#                                                    discretize_continuous=False)

# # explain a single instance using the LIME explainer
# exp = explainer.explain_instance(
#     X_test_poly[1], predict, num_features=20)

# # print the LIME explanation
# print(exp.as_list())

# # plot the LIME explanation
# exp.show_in_notebook()

# # style the LIME explanation using to_pyplot()
# fig = exp.as_pyplot_figure()
# axes = fig.get_axes()[0]  # get the axes object

# # adjust the spines
# axes.spines['top'].set_visible(False)
# axes.spines['right'].set_visible(False)
# axes.spines['bottom'].set_color('#DDDDDD')
# axes.spines['left'].set_color('#DDDDDD')
# axes.tick_params(axis='x', colors='#999999')
# axes.tick_params(axis='y', colors='#999999')
# axes.yaxis.label.set_color('#999999')
# axes.xaxis.label.set_color('#999999')